In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, fbeta_score, precision_recall_curve

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB

import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
df = pd.read_pickle('kickstarter_analysis.pkl')
df = df.dropna()

# Blurbs

In [3]:
y = df['failed'].values
words = set(nltk.corpus.words.words())

In [4]:
def non_eng(string):
    phrase = " ".join(w for w in nltk.wordpunct_tokenize(string) if w.lower() in words or not w.isalpha())
    return phrase

In [5]:
## PROCESSESING USING LEMMATIZATION
lemmatizer = WordNetLemmatizer() 
df['blurb_fix'] = df['blurb'].str.lower() ##lowercase
df['blurb_fix'] = df['blurb_fix'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x.split()]) ##get lemma
df['blurb_fix'] = df['blurb_fix'].apply(lambda x: ' '.join(x)) ##put into one string
df['blurb_fix'] = df['blurb_fix'].apply(non_eng) ##remove non-english words
df['blurb_fix'] = df['blurb_fix'].str.replace(r'[^\w\s]+', '') ## remove punctuation

In [6]:
vectorizer = TfidfVectorizer(min_df = 10, stop_words='english')
Xt = vectorizer.fit_transform(df['blurb_fix'])

In [7]:
Xt_train, Xt_test, yt_train, yt_test = train_test_split(Xt, y, 
                                                   test_size = 0.4, random_state = 42)

In [8]:
clf = MultinomialNB()
clf.fit(Xt_train, yt_train)

trnt = clf.predict(Xt_train)
tst = clf.predict(Xt_test)

train_score = fbeta_score(yt_train, trnt, beta = 1.1)
score = fbeta_score(yt_test, tst, beta = 1.1)

print("MNB Training Score: {:2f}% \nMNB Testing Score: {:.2f}%".format(train_score*100, score*100))

pd.crosstab(yt_test, tst, rownames=['True'], colnames=['Predicted'], margins=True)

MNB Training Score: 62.199790% 
MNB Testing Score: 58.29%


Predicted,0,1,All
True,,,
0,30053,8095,38148
1,13894,15751,29645
All,43947,23846,67793


In [9]:
df['MNB_Blurb_All'] = clf.predict_proba(Xt)[:,1]

# Blurb Categories

In [10]:
cats = list(df['category_core'].unique())

In [11]:
stored = {}

In [12]:
stored = {}
for c in cats:
     stored[c] = df[df['category_core'] == c]

In [13]:
for c in cats:
    data = stored[c]
    y = data['failed'].values
    
    vectorizer = TfidfVectorizer(min_df = 10, stop_words='english')
    X = vectorizer.fit_transform(data['blurb_fix'])

    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size = 0.4, random_state = 42)

    clf = MultinomialNB()
    clf.fit(X_train, y_train)

    stored[c]['MNB_Blurb_Cat'] = clf.predict_proba(X)[:,1]
    
    print("THE WORD IS {}".format(c))
    words = np.array(vectorizer.get_feature_names())
    classes = clf.classes_
    x = np.eye(X.shape[1])
    probs = clf.predict_log_proba(x)
    for i, c in enumerate(classes):
        prob = probs[:,i]
        ind = np.argsort(prob)[::-1]
    
        good_words = words[ind[:5]]
    
        good_prob = prob[ind[:5]]
        
        print(c,":")
        for w, p in zip(good_words, good_prob):
            print("{} | {:.2f}".format(w, np.exp(p)))

/Users/Matt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


THE WORD IS food
0 :
keto | 0.75
bitter | 0.72
knife | 0.71
butcher | 0.70
iconic | 0.69
1 :
smile | 0.92
wonderful | 0.91
al | 0.91
trying | 0.91
pork | 0.90


/Users/Matt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


THE WORD IS film & video
0 :
portrait | 0.93
documentary | 0.93
stretch | 0.92
funeral | 0.88
refugee | 0.88
1 :
healthy | 0.86
hey | 0.85
cartoon | 0.82
screenplay | 0.82
wanting | 0.81


/Users/Matt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


...

/Users/Matt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


THE WORD IS games
0 :
28mm | 0.97
novel | 0.95
miniature | 0.94
5e | 0.94
visual | 0.93
1 :
ball | 0.85
rush | 0.83
tag | 0.82
flag | 0.82
golf | 0.81


/Users/Matt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Matt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


THE WORD IS comics
0 :
issue | 0.96
collected | 0.95
collection | 0.93
high | 0.93
face | 0.93
1 :
writing | 0.67
martial | 0.59
web | 0.58
purpose | 0.57
store | 0.54
THE WORD IS dance
0 :
work | 0.96
collaboration | 0.96
concert | 0.96
length | 0.95
premiere | 0.95
1 :
confidence | 0.77
adult | 0.65
fit | 0.62
purpose | 0.60
para | 0.60
THE WORD IS design
0 :
watch | 0.95
leather | 0.95
pen | 0.95
pocket | 0.95
bag | 0.94
1 :
barn | 0.84
healing | 0.82
related | 0.79
retreat | 0.79
local | 0.78


/Users/Matt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
final = list()
for c in cats:
    final.append(stored[c])

In [15]:
final_all = pd.concat(final)

# Description

In [16]:
df = final_all

In [17]:
y = df['failed'].values
words = set(nltk.corpus.words.words())

In [18]:
## PROCESSESING USING LEMMATIZATION
lemmatizer = WordNetLemmatizer() 
df['desc_fix'] = df['description'].str.lower() ##lowercase
df['desc_fix'] = df['desc_fix'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x.split()]) ##get lemma
df['desc_fix'] = df['desc_fix'].apply(lambda x: ' '.join(x)) ##put into one string
df['desc_fix'] = df['desc_fix'].apply(non_eng) ##remove non-english words
df['desc_fix'] = df['desc_fix'].str.replace(r'[^\w\s]+', '') ## remove punctuation

In [19]:
vectorizer = TfidfVectorizer(min_df = 10, stop_words='english')
Xt = vectorizer.fit_transform(df['desc_fix'])

In [20]:
Xt_train, Xt_test, yt_train, yt_test = train_test_split(Xt, y, 
                                                   test_size = 0.4, random_state = 42)

In [21]:
clf = MultinomialNB()
clf.fit(Xt_train, yt_train)

trnt = clf.predict(Xt_train)
tst = clf.predict(Xt_test)

train_score = fbeta_score(yt_train, trnt, beta = 1.1)
score = fbeta_score(yt_test, tst, beta = 1.1)

print("MNB Training Score: {:2f}% \nMNB Testing Score: {:.2f}%".format(train_score*100, score*100))

MNB Training Score: 64.203981% 
MNB Testing Score: 61.68%


In [22]:
df['MNB_Desc_All'] = clf.predict_proba(Xt)[:,1]

## Description Categories

In [23]:
cats = list(df['category_core'].unique())

In [24]:
stored = {}
for c in cats:
     stored[c] = df[df['category_core'] == c]

In [ ]:
for c in cats:
    data = stored[c]
    y = data['failed'].values
    
    vectorizer = TfidfVectorizer(min_df = 10, stop_words='english')
    X = vectorizer.fit_transform(data['desc_fix'])

    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size = 0.4, random_state = 42)

    clf = MultinomialNB()
    clf.fit(X_train, y_train)

    stored[c]['MNB_Desc_Cat'] = clf.predict_proba(X)[:,1]
    
    print("THE WORD IS {}".format(c))
    words = np.array(vectorizer.get_feature_names())
    classes = clf.classes_
    x = np.eye(X.shape[1])
    probs = clf.predict_log_proba(x)
    for i, c in enumerate(classes):
        prob = probs[:,i]
        ind = np.argsort(prob)[::-1]
    
        good_words = words[ind[:5]]
    
        good_prob = prob[ind[:5]]
        
        print(c,":")
        for w, p in zip(good_words, good_prob):
            print("{} | {:.2f}".format(w, np.exp(p)))

/Users/Matt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


THE WORD IS food
0 :
unlocked | 0.66
birch | 0.65
keto | 0.64
bitters | 0.64
ketogenic | 0.63
1 :
ich | 0.94
franchise | 0.89
ist | 0.89
homeless | 0.89
da | 0.88


/Users/Matt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


THE WORD IS film & video
0 :
archival | 0.87
documentary | 0.87
refugee | 0.85
deductible | 0.85
correction | 0.85
1 :
marshal | 0.77
twitch | 0.75
ich | 0.75
cartoon | 0.75
ruler | 0.74


In [47]:
alldesc = list()
for c in cats:
    alldesc.append(stored[c])
    
alldesc = pd.concat(alldesc)

In [48]:
alldesc.head()

,blurb,country,created_at,currency,deadline,launched_at,slug,spotlight,staff_pick,state,...,created_weekday,deadline_month,deadline_weekday,length_of_project,blurb_fix,MNB_Blurb_All,MNB_Blurb_Cat,desc_fix,MNB_Desc_All,MNB_Desc_Cat
0,Ginger KICK! is back for the holidays with ama...,US,2017-11-05 12:05:03,USD,2017-11-12 20:33:24,2017-11-05 20:33:24,ginger-kick-holiday-cheer,1,0,successful,...,Sun,Nov,Sun,7,ginger kick is back for the holiday with amaz...,1,0,for those visiting for the first time here s...,0,0
11,More than just a farm.,CA,2015-06-29 03:23:38,CAD,2015-12-05 06:00:00,2015-11-05 15:02:36,flavourful-farms,1,1,successful,...,Mon,Dec,Sat,159,more than just a farm,1,1,built to serve you farm wa after year of resea...,1,1
13,"I'm on a mission to make the best, most sinful...",US,2014-07-07 21:05:23,USD,2014-09-09 03:44:57,2014-07-11 03:44:57,creating-a-perfect-mac-and-cheese,0,0,failed,...,Mon,Sep,Tue,63,i m on a mission to make the best most sinfu...,0,1,my goal is to create ultra gourmet mac and che...,1,1
14,I want to open a neighborhood bakery so I can ...,US,2015-08-14 00:29:00,USD,2015-09-14 11:21:58,2015-08-15 11:21:58,old-irish-bakery,0,0,failed,...,Fri,Sep,Mon,31,i want to open a neighborhood bakery so i can ...,1,1,every one of my happy childhood memory involve...,1,1
18,A wine lounge that educates the mind and the p...,US,2018-02-02 02:26:14,USD,2018-05-20 03:59:00,2018-04-19 17:32:16,blacksburg-wine-lab,1,0,successful,...,Fri,May,Sun,107,a wine lounge that the mind and the palate in...,1,1,overview wine lab is so because it will be de...,1,1


In [49]:
alldesc.to_pickle('kickstarter_NB.pkl')